Imports de pacotes do Python

In [4]:
import sys
from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


# NO CASO ESTÁ IMPORTANDO A CNN VGG16. CONSULTA NA INTERNET tensorflow.keras.applications
# POIS HÁ VARIAS OUTRAS CNNS QUE VC PODE TESTAR.
from tensorflow.keras.applications import VGG16


Monta o Google Drive

In [5]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
# define cnn model using VGG16
def define_model(num_classes, input_shape):

    # PARA TESTAR OUTRAS CNNS, ALÉM DA ALTERAÇÃO DA IMPORTAÇÃO DESSA ACIMA,
    # VC VAI TROCAR VGG16 PELO NOME DA CNN IMPORTADA

    # Load the VGG16 model, excluding the top (fully connected) layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Create a new model
    model = Sequential()
    # Add the VGG16 base model
    model.add(base_model)
    # Flatten the output from the base model
    model.add(Flatten())
    # Add new fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    # Output layer with num_classes units and softmax activation
    model.add(Dense(88, activation='softmax'))

    # Compile the model with categorical crossentropy for multi-class classification
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

# Funcao principal
def funcao_principal(num_classes, base_path, input_shape):
    # chama a função que define o modelo da rede
    model = define_model(num_classes, input_shape)

    # cria o gerador de dados de treino fazendo a normalização dos valores de pixels na escala real [0,1] e tirando 30% das amostras para validacao
    datagen_train = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.3)

    # cria o gerador de dados de teste fazendo a normalização dos valores de pixels na escala real [0,1]
    datagen_test = ImageDataGenerator(rescale=1.0/255.0)

    # prepara os iteradores
    batch_size = 32

    # SE VOCÊ TIVER COM CONTA COLAB PRO, O CERTO É ZIPAR, BAIXAR E DESCOMPACTAR OS ARQUIVOS PARA A MAQUINA.
    # PELO Drive O ACESSO É LENTO
    train_it = datagen_train.flow_from_directory(os.path.join(base_path, 'train'),
        class_mode='categorical', batch_size=batch_size, target_size=(200, 200))

    validation_it = datagen_train.flow_from_directory(os.path.join(base_path, 'validation'),
        class_mode='categorical', batch_size=batch_size, target_size=(200, 200))

    test_it = datagen_test.flow_from_directory(os.path.join(base_path,'test'),
        class_mode='categorical', target_size=(200, 200))

    # fit model
    history = model.fit(train_it, validation_data=validation_it,
        validation_steps=validation_it.samples//batch_size, epochs=20, verbose=1)

Chama a função principal

In [ ]:
# Aciona o programa
input_shape = (200, 200, 3)


base_path  = '/content/drive/MyDrive/curso-ia-zootecnia/aula-03/dogs_breed'

nclasses = len([name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, 'train'))])


funcao_principal(88, base_path, input_shape)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 10413 images belonging to 88 classes.
Found 2985 images belonging to 88 classes.
Found 1534 images belonging to 88 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 92/326 ━━━━━━━━━━━━━━━━━━━━ 42:48 11s/step - accuracy: 0.0106 - loss: 4.5644

Avalia o Modelo

In [ ]:

# evaluate model
_, acc = model.evaluate_generator(test_it, verbose=1)
print('> %.3f' % (acc * 100.0))

# learning curves
summarize_diagnostics(history)